### Imports

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

from utils.loader import llm, emb_fn, strings
from utils.utils import prepare_data, set_all_seeds, save_predictions, calculate_metrics
from utils.optimizer import Protegi

In [2]:
set_all_seeds(42)

### Metrics

In [66]:
accuracy = accuracy_score(df['v1'], df['predictions'])
recall = recall_score(df['v1'], df['predictions'])
f1 = f1_score(df['v1'], df['predictions'])
precision = precision_score(df['v1'], df['predictions'])
print(f'Accuracy = {accuracy:.2f}\nRecall = {recall:.2f}\nPrecision = {precision:.2f}\nF1 = {f1:.2f}')

Accuracy = 0.82
Recall = 1.00
Precision = 0.40
F1 = 0.57
